# Predefinisani projekat za 60 bodova
##### Autor: Stevan Rašković RA113/2015
<hr/> 
### Opis problema koji se rešava:
Na video zapisima se nalaze zelena i plava linija, kao i cifre koje prolaze ispod njih. Ukoliko cifra prođe ispod plave linije treba je dodati ukupnoj sumi, a ukoliko prođe ispode zelene linije treba je oduzeti od sume.

![](img/videos.png)

### Postupak rešavanje problema:
- Učitavanje video zapisa kao niza frejmova
- Detekcija linija
- Detekcija regiona koji sadrže cifre
- Praćenje regiona u odnosu na prethodni frejm
- Ispitivanje prolaska regiona koji se prate ispod neke od linija
- Upotreba neuronske mreže za prepoznavanje cifre koja se nalayi unutar regiona

<hr/>
#### Uključivanje potrebnih modula:

In [15]:
import os
import tensorflow as tf
import numpy as np
import cv2 
from collections import OrderedDict
import scipy
from scipy.stats import norm
from collections import OrderedDict
from scipy.spatial import distance as dist
from pathlib import Path  
import matplotlib.pyplot as plt  
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import optimizers
from keras.datasets import mnist
from keras.layers import Dropout 

<hr/>
#### Funkcija za pronalaženje linije na frejmu:
Ulazni parametri funkcije su slika na kojoj se traži linija i indeks kanala boje na kom se traži (R=0 G=1 B=2)
Koraci za detekciju linije koje funckija primenjuje su sledeči:
- Postavljanje svih kanala osim prosleđenog na 0
- Primena morfološke operacije otvaranja
- Detekcija ivica Canny algoritmom
- Primena Hough transformacije za detekciju linija
- Uprosečavanje izlaza Hough transformacije, kako bi se dobila samo jedna linija

In [16]:
def findLine(image,channel):
    img=image.copy()
    #Postavljanje svih kanala osim onog po kom trazimo na 0
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    for i in range(3):
        if i!=channel:
            img[:,:,i]=0
    #Otvaranje
    ker=cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    img=cv2.morphologyEx(img, cv2.MORPH_OPEN, ker)
    
    #Canny
    img = cv2.Canny(img, 200, 250)
    img = cv2.dilate(img, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    
    #Hough
    lines = cv2.HoughLinesP(img, 1, np.pi / 180, 50, None, 180, 50) 
    
    #posto hough transformacija moze da vrati vise linija, trazimo njihov prosek
    lines=lines.tolist()
    x1,y1,x2,y2=0,0,0,0
    for i in range(0,len(lines)):
        x1+=lines[i][0][0]
        y1+=lines[i][0][1]
        x2+=lines[i][0][2]
        y2+=lines[i][0][3]
    x1/=len(lines)
    x2/=len(lines)
    y1/=len(lines)
    y2/=len(lines)
    return [int(x1),int(y1),int(x2),int(y2)]

<hr/>
#### Pomoćne funkcije za rad sa slikom:
##### Promena veličine slike:

In [17]:
def scale(image,x,y,inter=cv2.INTER_NEAREST):
    return cv2.resize(image,(x,y), interpolation = inter)

##### Uklanjanje crnog okvira sa slike:

In [18]:
def removeBlackPading(image):
    white = image > 0
    return image[np.ix_(white.any(1),white.any(0))]

##### Izmena MNIST skupa za treniranje neuronske mreže:
Koraci koji su primenjeni na svaki element su:
- Uklanjanjnje crnog okvira (jer će se od mreže očekivati da prepoznaje brojeve koji nemaju crni okvir)
- Skaliranje na dimnezije 28*28
- Normalizacija na opseg [0,1]

In [19]:
def prepareForNN(set):
    x=y=28
    ret = np.empty([len(set), x, y])
    i=0
    for image in set:
        img=removeBlackPading(image)
        ret[i]=scale(img,x,y)
        i=1+i
    ret = ret.astype('float32')
    ret /= 255.0
    return ret

### Funkcija za kreiranje neuronske mreže
Ukoliko postoji fajl sa sačuvanim putanjama na prosleđenoj putanji, težine će biti učitane <br/>
U suprotnom će biti istrenirana nova mreža i težine će se sačuvati u prosleđeni fajl

In [20]:
def getNeuralNetwork(file):
   
    neuralNetwork = Sequential()
    neuralNetwork.add(Dense(512,input_shape=(784,), activation = 'relu'))
    neuralNetwork.add(Dropout(0.2))
  
    neuralNetwork.add(Dense(512, activation = 'relu'))
    neuralNetwork.add(Dropout(0.2))
    neuralNetwork.add(Dense(10, activation = 'softmax'))
    if Path(file).is_file():
        neuralNetwork.load_weights(file)
    else: 
        (trainSetX, trainSetY), (testSetX, testSetY) = tf.keras.datasets.mnist.load_data()
        trainSetX= prepareForNN(trainSetX)
        testSetX=prepareForNN(testSetX)

        testSetX=testSetX.reshape(10000,784)
        trainSetX=trainSetX.reshape(60000,784)
        ##testSetY = tf.keras.utils.to_categorical(testSetY, 10)
        #trainSetY = tf.keras.utils.to_categorical(trainSetY, 10)
       
        neuralNetwork.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        neuralNetwork.fit(trainSetX, trainSetY, batch_size=200, epochs=34, verbose=1)
        corr=neuralNetwork.evaluate(testSetX, testSetY)
        neuralNetwork.save_weights(file)
    return neuralNetwork

<hr/>
### Funkcija zadužena za pronalaženje cifara na slici
Koraci:
- Pretvaranje slike u sivu
- Binarizacija
- Pronalaženje regiona
- Sortiranje regiona po x-osi

![](img/detection.png)

In [21]:
def findNumbers(image):
    image=image.copy()
    img=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #siva
    _, img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY) #binarna
    contours, hierarchy = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) #iydvajanje kontura
    
    coordinates = []
    regions = []
    
    for i in range(0, len(contours)):
        contour = contours[i]    
        x, y, w, h = cv2.boundingRect(contour) #(x,y) gornji levi cosak, w sirina, h visina
        area = cv2.contourArea(contour)
        if(h > 14 and h <= 26) or (w >= 11 and h > 13) and area < 808 and (hierarchy[0][i][3] == -1):
         
            region = img[y:y+h+1, x:x+w+1]
            regions.append([scale(region,28,28), x])       
            coordinates.append([x, y, x+w, y+h])
           #cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    #sort
    coordinates.sort(key=lambda tup: tup[0])
    regions = sorted(regions, key=lambda item: item[1])
    regions = [region[0] for region in regions]
    return regions, coordinates

<hr/>
### Funckija koja određuje da li je cifra prošla ispod linije:
Parametri funkcije su koordinate linije i koordinate regiona u kom se nalazi cifra
Za prolnalaženje razdaljine koriste se osobina vektorsog prozivoda da njegova apsolutna vrednost predstavalja površinu paralelorgama konstruisanog nad vektorima od kojih je nasao.
![](img/cross.png)<br/>
Detaljnije objašnjenje i vizuelizacija može se naći [ovde](https://www.youtube.com/watch?v=tYUtWYGUqgw)

Nakon određivanja udaljenosti, proverava se da li je udaljenost manja od unapred zadatog parametra

In [22]:
def crossLine(line, coordinates):
    if line[0] >=coordinates[2] or line[2] <= coordinates[0]:
        return False
    lineStart = np.array([line[0], line[1]]) #tacka na kojoj linija pocinje
    lineEnd = np.array([line[2], line[3]]) #tacka na kojoj se linija zavrsava
    point = np.array([coordinates[2], coordinates[3]]) #tacka za koju se trazi rastojanje
    distance = abs(np.cross(lineEnd-lineStart, point-lineStart)/np.linalg.norm(lineEnd-lineStart)) #rastoranje
    
    if (distance < 3):   
        return True
    return False

<hr/>
#### Klasa koja služi za praćenje kontura
Osnovna ideja i delovi algoritma su preuzeti sa [ovog](https://www.pyimagesearch.com/2018/07/23/simple-object-tracking-with-opencv/) linka<br/>
Klasa poseduje rečnik centara kontura koje prati, kao i rečnik objekata klase Number
Glavna metoda ovde klase koja se koristi za praćenje je metoad update, koja prima niz ulaznih kontura
Koraci koji se primenjuju prilikom praćenja:
- Izračunavanje centara kontura
- Računanje euklidskog rastojanja između novih kontura i kontura koje su do sada registrovane u klasi
- Ažuritanje postjećih kontura novim vrednostima, pri čemu se stara kontura ažurira koordinatama najbliže nove konture
- Registrovanje novih kontura, koje nisu povezane ni sa jednom prethodno registrovanom
- Brisanje kontura koje nisu dugo ažurirane

![](img/trackerv2.png)

In [23]:
class Tracker():
    #Konstruktor: pravi prazne recnike, inicijalizuje id
    def __init__(self):
        self.numbers = dict() #<idObjekta,Number>
        self.nextID = 0 #sledeci id 
        self.centers=dict() #koordinate centara
        self.deleted=dict() #izbrisani
    #Metoda za dodavanje nove konture
    def insert(self,center, coordinates,image):
        number=Number(coordinates,image)
        self.numbers[self.nextID]=number
        self.centers[self.nextID]=center
        self.nextID += 1

    #Uklanjanje konture
    #Proveravamo da li postoji cifra 
    def remove(self, id):
        del self.centers[id]
        num=self.numbers.pop(id)
        for delId in self.deleted.keys():
            if abs(self.deleted[delId].image-num.image).sum()<100:
                self.deleted[delId].coordinates=num.coordinates
                return
        self.deleted[id]=num
        
        
    #Peacaenje
    def update(self, inputCts,images):
        #Ako nema ulaznih kontura, svim konturama povecamo broj frejmova koji su bile nevidljive i izbrisemo one koje su prekoracile limit
        if len(inputCts) == 0:
            forRemove=[]
            for numID in self.numbers.keys():
                self.numbers[numID].lastSeen=self.numbers[numID].coordinates
                self.numbers[numID].invisibleFor += 1
                if self.numbers[numID].invisibleFor > 60: 
                    forRemove.append(numID)
            for a in forRemove:       
                self.remove(a)
            return self.numbers

        #Ako imamo ulazne konture, trazimo centre svih ulaznih kontura
        centroids = np.zeros((len(inputCts), 2), dtype="int")
        for (i, (x, y, w, h)) in enumerate(inputCts):
            centerX= int((x + w) / 2.0)
            centerY = int((y + h) / 2.0)
            centroids[i] = (centerX, centerY)
      
        #Ako nemamo nijednu registrovanu, sve ih dodajemo
        if len(self.numbers) == 0:
            for i in range(0, len(inputCts)):
                self.insert(centroids[i], inputCts[i],images[i])

        #Ako postoje konture koje imamo registrovane od ranije, pokusavamo nove konture da povezemo sa njim
        else:
            IDs = list(self.centers.keys())
            centerVals = list(self.centers.values())
            #Trazimo euklidsko rastojanje novih i postojecih kontura
            distance = dist.cdist(np.array(centerVals), centroids)
            
            #Sortiramo redove na osnovu najmanje udaljenosti
            rows = distance.min(axis=1).argsort()

            #Sortiramo kolone u redovima
            cols = distance.argmin(axis=1)[rows]

            #Skupovi redova i kolona koje su vec upotrebljene
            usedRows = set()
            usedCols = set()

            #Iteriramo kroz uredjenje parove (red,kolona)
            for (row, col) in zip(rows, cols):
                #Ako je neka komponenta uredjenog para iskoristena, nastavljamo dalje
                if row in usedRows or col in usedCols:
                    continue
                #Ako nije, azuriramo konturu sa indeksom row podacima sa indeksom col
                id = IDs[row]
                self.centers[id] = centroids[col]
                self.numbers[id].invisibleFor = 0
                self.numbers[id].coordinates = inputCts[col]
                self.numbers[id].image=images[col]
                # Belezimo da smo iskoristili red i kolonu
                usedRows.add(row)
                usedCols.add(col)

            #Uzimamo sve redove i kolone koje nisu upotrebljene
            unusedRows = set(range(0, distance.shape[0])).difference(usedRows)
            unusedCols = set(range(0, distance.shape[1])).difference(usedCols)

            #Ako imamo vise registrovanih kontura nego sto ih ima sa ulaznog frejma, 
            #Tim kontuirama povecavamo broj frejmova od poslednjeg puta kad smo ih videli
            #I po potrebi ih brisemo
            if distance.shape[0] >= distance.shape[1]:
                forRemove=[]
                for row in unusedRows:
                    id = IDs[row]
                    self.numbers[id].invisibleFor += 1
                    if self.numbers[id].invisibleFor > 60:
                        forRemove.append(id)
                for a in forRemove:        
                    self.remove(a)
            
            #Ako postoje ulazne konture koje nismo povezali ni sa jednom prethodno registrovanom, onda je registrujemo kao novu
            else:
                for col in unusedCols:
                    self.insert(centroids[col], inputCts[col],images[col])
        return self.numbers
    
class Number:
    def __init__(this,coordinates,image):
        this.invisibleFor=0 #koliko frejmova se region nuje video
        this.image=image #slika regiona
        this.firstSeen=coordinates #mesto gde je vidjen prvi put
        this.coordinates=coordinates #poslednji put

<hr/>
#### Funkcije za rešavanje problema sa preklopljenim ciframa
Pošto se zbog preklapanja može desiti da se ne detektuje prelazak neke cifre preko linije, na kraju svakog videa se za sve konture koje su detektovane a nisu registrovane da su prošle ispod neke od linija, proverava da li linija koja prolazi kroz mesta gde je prvi i poslednji put detektovana seče zelenu ili plavu liniju. Ukoliko seče, neuronska mreža prepoznaje cifru i ona se uzima u obzir prilikom računanja krajnjeg rezultata.

![](img/over.png)

In [24]:
def overlap(items,add,substract,lineGreen,lineBlue,nn):
    kplus=(lineBlue[1]-lineBlue[3])/(lineBlue[0]-lineBlue[2]) #koeficijent pravca plave
    nplus=lineBlue[1]-kplus*lineBlue[0] 
    kminus=(lineGreen[1]-lineGreen[3])/(lineGreen[0]-lineGreen[2]) #koeficijent pravca zelene
    nminus=lineGreen[1]-kminus*lineGreen[0]  
    for itemID in items.keys():
        #break
        item=items[itemID]
        if itemID not in add and itemID not in substract:   
            num=overlapHelper(item,kplus,nplus,lineBlue,nn)
            if num is not None:
                add[itemID]=num
            num=overlapHelper(item,kminus,nminus,lineGreen,nn)
            if num is not None:
                substract[itemID]=num

def overlapHelper(item,lineK,lineN,line,nn):
    n1=item.coordinates[3]-lineK*(item.coordinates[2]) #paralela
    n2=item.firstSeen[3]-lineK*(item.firstSeen[2]) #paralela
    if (n1-lineN)*(n2-lineN)<0:  # da li su paralele sa razliiitih strana linije?
        k=(item.firstSeen[3]-item.coordinates[3])/(item.firstSeen[2]-item.coordinates[2])
        nnp=item.firstSeen[3]-k*item.firstSeen[2]
        xi=(nnp-lineN)/(lineK-k)
        corr=item.firstSeen[0]-item.firstSeen[2]
        corr=abs(corr)*1.0
        if xi-corr>=line[0] and xi<=line[2]+corr:
            forNN = prepareNumber(item.image)
            result = nn.predict(np.array([forNN]))
            r = result.reshape(1,10)
            ret = int(np.argmax(r))  
            return ret
        return None

<hr/>
#### Funckija koja računa sumu za prosleđeni video
Ova funkcija kao parametre prima putanju do ulaznog video zapisa kao i neuronsku mrežu koje če koristiti za prepoznavanje cifre
Koraci:
- Učitavanje snimka kao niz frejmove
- Pronalaženje linija
- Praćenje kontura
- Detekcija prolaska cifre ispod linije
- Prepoznavanje cifre upotrebom neuronske mreže
- Dodavanje prepoznate cifre u rečnik cifara koje treba oduzeti/dodati
- Dodavanje cifara za koje zbog preklapanja nije detektovan prelazak tokom praćenja kontura

Funckija ne vrši momentalno dodavanje prepoznate cifre na konačnu sumu, već broj koji treba dodati čuva u rečniku a kao ključ koristi id konture sa koje je cifra prepoznata. Nakon obrade frejma, za sve konture koje se ne nalaze u rečnicima za sabiranje i oduzimanje, vrši se korekcija rezultata na način koji je opisan u prethodnom segmentu.

In [25]:
def calculate(path,nn):
    add=dict()
    tracker=Tracker()
    substract=dict()
    video = cv2.VideoCapture(path)
    lineGreen=[]
    lineBlue=[]
    sum=0
    while(video.isOpened()):
        ret, frame = video.read()
        if ret == True:
            lineGreen=findLine(frame,1)
            lineBlue=findLine(frame,2)
            regions, coordinates=findNumbers(frame)
            numbers=tracker.update(coordinates,regions)
            #cv2.imshow("v",img) 
            #if cv2.waitKey(1) & 0xFF == ord('q'):
             #   break
                
            IDs = list(numbers.keys())
            for id in IDs:
                number = numbers[id]
                if id not in add:
                    if crossLine(lineBlue, number.coordinates):
                        add[id]=getNumberFromRegions(number,coordinates,regions,nn)

                if id not in substract:
                    if crossLine(lineGreen, number.coordinates):
                        substract[id]=getNumberFromRegions(number,coordinates,regions,nn)
        
        else:
            break        


    kplus=(lineBlue[1]-lineBlue[3])/(lineBlue[0]-lineBlue[2])
    nplus=lineBlue[1]-kplus*lineBlue[0] 
    
    kminus=(lineGreen[1]-lineGreen[3])/(lineGreen[0]-lineGreen[2])
    nminus=lineGreen[1]-kminus*lineGreen[0]  
 
    overlap(tracker.deleted,add,substract,lineGreen,lineBlue,nn)
    overlap(tracker.numbers,add,substract,lineGreen,lineBlue,nn) 

    for idObjekta in add:
        if idObjekta not in substract:
            sum+=add[idObjekta]

    for idObjekta in substract:
        if idObjekta not in add:
            sum-=substract[idObjekta]
    return sum

#pronalazi region sa prosledjenim koordinatama i vrsi predikciju za njega
def getNumberFromRegions(number,coordinates,regions,nn):
    ind=-1
    for i in (range(0, len(coordinates))):
        if coordinates[i][0] == number.coordinates[0] and coordinates[i][1] == number.coordinates[1]:
            ind=i
            break
    forNN = prepareNumber(regions[ind])
    result = nn.predict(np.array([forNN]))
    for r in result:
        r = r.reshape(1,10)
        ret = int(np.argmax(r))   
    return ret
#radi zatvaranje i pripremu regiona za neuronsku mrezu
def prepareNumber(region):
    ker=cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    region=cv2.morphologyEx(region, cv2.MORPH_CLOSE, ker)
    region=region.astype('float32')/255.0
    region=region.flatten()
    return region


<hr/>
### Glavna funckija
U ovoj funciji se iterira kroz sve zapise, vrši se računanje sume i ispis svih suma u fajl

In [26]:
def doAll():
    f= open("out.txt","w+")
    f.write("RA 113/2015 Stevan Raskovic\n")
    f.write("file\tsum\n")
    nn=getNeuralNetwork("neuralNetwork.h5")
    for i in range(0,10):
        print("video "+str(i))
        name="video-"+str(i)+".avi"
        s=calculate("videos/"+name,nn)
        f.write(name+'\t'+str(s)+'\n')
        print("suma "+str(s))
        print("#######")
    f.close()


In [27]:
doAll()
exec(open('test.py').read())

video 0
suma -32
#######
video 1
suma -18
#######
video 2
suma -3
#######
video 3
suma -60
#######
video 4
suma -40
#######
video 5
suma 24
#######
video 6
suma -64
#######
video 7
suma 9
#######
video 8
suma -3
#######
video 9
suma 21
#######
['RA 113/2015 Stevan Raskovic']
Procenat tacnosti:	79.51807228915663
Ukupno:	10
